In [1]:
#!pip3 install pandas openai langchain langchain_community rapidocr-onnxruntime pypdf faiss-cpu langchain_openai

In [40]:
import os
import openai
import pandas as pd

from datetime import datetime

from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_community.llms import LlamaCpp
from langchain.prompts.prompt import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
# from questions import questions
# from answers import responses
# from helper import (
#     text_to_csv,
#     get_time_difference,
#     filter_csv,
#     read_csv,
#     read_counter,
#     increment_counter,
#     text_to_file,
# )
from secret import OPENAI_API_KEY

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
embeddings = OpenAIEmbeddings(openai_api_key=os.environ["OPENAI_API_KEY"])
vector_store_path = "./vector_store_tt"
openai.api_key = os.environ["OPENAI_API_KEY"]

In [41]:
def create_vector_store(file_path):
    loader = PyPDFLoader(file_path, extract_images=True)
    
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=20,
        keep_separator=False,
        separators=["\n\n", "\n", ""],
    )
    
    splitted_documents = loader.load_and_split(text_splitter=text_splitter)
    
    # splitted_documents = text_splitter.split_documents(documents)
    
    vectordb = FAISS.from_documents(splitted_documents, embedding=embeddings)
    vectordb.save_local(vector_store_path)
    
    return splitted_documents

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


def get_retriever():
    vectorstore = FAISS.load_local(
        vector_store_path, embeddings, allow_dangerous_deserialization=True
    )

    retriever = vectorstore.as_retriever(search_kwargs={"k": 20})

    return retriever

In [44]:
doc_name = "Doc-8"
file_path = f"../docs/{doc_name}.pdf"

In [45]:
docs= create_vector_store(file_path)

In [46]:
llm = ChatOpenAI(
    model="gpt-4-0125-preview",  # "ft:gpt-3.5-turbo-1106:mobilelive-inc:irap-model:8mTN1w61",
    openai_api_key=os.environ["OPENAI_API_KEY"],
    temperature=0.5,
)

retriever = get_retriever()

In [47]:
from answers import responses

In [48]:
def get_prompt(template=None):
    if template is None:
        template = """You are an AI assistant to answer question as per the given context. 
        Use the following pieces of retrieved context to answer the question. 
        Please provide answer as per the given context.
      Question: {question}
      Context: {context}
      Answer:
      """

    prompt_template = PromptTemplate(
        template=template,
        input_variables=[
            "context",
            "question",
        ],
    )

    return prompt_template

In [49]:
import os 

folder_path = f"./output/{doc_name}"

if not os.path.exists(folder_path):
    os.makedirs(folder_path)

In [106]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

# Define your desired data structure.
class WbsModel(BaseModel):
    Level: str = Field(description="Serial Number")
    Category: str = Field(description="Category of Task")
    Subcategory: str = Field(description="Sub-Category of Task")
    UserStory: str = Field(description="User story related to Task")
    EffortsInDays: str = Field(description="Time estimations")

parser = JsonOutputParser(pydantic_object=WbsModel)

In [107]:
def get_prompt():
    template = f"""You are an AI assistant which is acting as a Technical Project Manager. 
    Provide the required information to the best of your knowledge.

    Please generate JSON-formatted data containing the following properties:
     "Level,Category,Subcategory,User Story,Effort (Days)"
    Ensure that the JSON format adheres to standard conventions.

    JSON_FORMAT:
        'Level': 1,
        'Category': 'Analysis',
        'Subcategory': 'Requirements Gathering',
        'User Story': 'Gather user requirements for the mobile application',
        'Effort (Days)': 2

    OBJECTIVE:
    The objective is to develop a structured and informative WBS that will serve as a clear roadmap for project execution.

    NOTE:
    Your answer should only include JSON data in the given format nothing else. Don't include ```json at begin and end of answer.

    CONTEXT:
    Consider the following sets of Questions and Answers for context:\n"""
    for question, response in responses.items():
        template += f"QUES: {question.strip()}\nANS: {response.strip()}\n\n"

    template +="""
      Question: {question}
      Context: {context}
      {format_instructions}
      Answer:
      """

    prompt_template = PromptTemplate(
        template=template,
        input_variables=[
            "context",
            "question",
        ],
        partial_variables={"format_instructions": parser.get_format_instructions()},
    )

    return prompt_template

In [108]:
prompt = get_prompt()

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | parser
)

In [111]:
final_wbs = []

In [112]:
query = """TASK:
    Your task is to create a comprehensive list of user stories covering all major aspects for UI/UX Development with technical details as well. 
    Please provide atleast 1-2 detailed requirements. Keep CATEGORY column value as DESIGN.
    Additionally, provide the estimate the effort (in days) to the best of your knowledge, with some buffer required for each requirement.
   """

first_pass = rag_chain.invoke(query)
final_wbs.extend(first_pass)
print(final_wbs)

[{'Level': '1', 'Category': 'DESIGN', 'Subcategory': 'UI/UX Development', 'User Story': 'Design intuitive and user-friendly interfaces suitable for both iOS and Android devices, ensuring consistent UI/UX across different screen sizes and resolutions.', 'Effort (Days)': '15'}, {'Level': '2', 'Category': 'DESIGN', 'Subcategory': 'UI/UX Development', 'User Story': 'Implement extensive customization features allowing users to personalize their experience, including themes, layouts, and interactive elements to enhance user engagement.', 'Effort (Days)': '10'}, {'Level': '3', 'Category': 'DESIGN', 'Subcategory': 'Accessibility and Localization', 'User Story': 'Ensure the application supports multiple languages and incorporates accessibility features to accommodate users with disabilities, including screen readers and voice navigation.', 'Effort (Days)': '12'}, {'Level': '4', 'Category': 'DESIGN', 'Subcategory': 'Performance Optimization', 'User Story': 'Optimize UI elements and animations to

In [115]:
d = [{'Level': '1', 'Category': 'DESIGN', 'Subcategory': 'UI/UX Development', 'UserStory': 'Design the login and user registration interface ensuring intuitive user interaction.', 'EffortsInDays': '5'}, {'Level': '2', 'Category': 'DESIGN', 'Subcategory': 'UI/UX Development', 'UserStory': 'Create a dashboard layout that dynamically displays KPIs, project estimations, and resource mix suggestions.', 'EffortsInDays': '8'}, {'Level': '3', 'Category': 'DESIGN', 'Subcategory': 'UI/UX Development', 'UserStory': 'Design a detailed profile view for resources including links to LinkedIn profiles and credit scores.', 'EffortsInDays': '4'}, {'Level': '4', 'Category': 'DESIGN', 'Subcategory': 'UI/UX Development', 'UserStory': 'Develop a system for displaying notifications, profiles, and settings in a user-friendly manner.', 'EffortsInDays': '6'}, {'Level': '5', 'Category': 'DESIGN', 'Subcategory': 'UI/UX Development', 'UserStory': 'Implement a role-based access control UI for managing user permissions and access to sensitive data.', 'EffortsInDays': '7'}, {'Level': '6', 'Category': 'DESIGN', 'Subcategory': 'UI/UX Development', 'UserStory': 'Design the UI for data retrieval from JIRA, including displaying Epics, Stories, and Tasks with their details.', 'EffortsInDays': '10'}, {'Level': '7', 'Category': 'DESIGN', 'Subcategory': 'UI/UX Development', 'UserStory': 'Create a user-friendly interface for the code reusability feature, showing percentage of reusable code.', 'EffortsInDays': '5'}, {'Level': '8', 'Category': 'DESIGN', 'Subcategory': 'UI/UX Development', 'UserStory': 'Develop an interface for project timeline estimation based on input from the Business Requirement Document.', 'EffortsInDays': '6'}, {'Level': '9', 'Category': 'DESIGN', 'Subcategory': 'UI/UX Development', 'UserStory': 'Design a feature for project costing that considers the suggested resource profile and estimated timeline.', 'EffortsInDays': '7'}, {'Level': '10', 'Category': 'DESIGN', 'Subcategory': 'UI/UX Development', 'UserStory': "Implement a UI component for integrating the platform with Bamboo HR to display employees' details.", 'EffortsInDays': '5'}, {'Level': '11', 'Category': 'DESIGN', 'Subcategory': 'UI/UX Development', 'UserStory': 'Design the interface for the credit scoring app, including plugin development for JIRA, GitHub, and SonarQube.', 'EffortsInDays': '9'}, {'Level': '12', 'Category': 'DESIGN', 'Subcategory': 'UI/UX Development', 'UserStory': 'Create a detailed UI for displaying project timelines and costing features, including labor and infrastructure expenses.', 'EffortsInDays': '8'}]

In [117]:
import json

d = json.dumps(d)
d

'[{"Level": "1", "Category": "DESIGN", "Subcategory": "UI/UX Development", "UserStory": "Design the login and user registration interface ensuring intuitive user interaction.", "EffortsInDays": "5"}, {"Level": "2", "Category": "DESIGN", "Subcategory": "UI/UX Development", "UserStory": "Create a dashboard layout that dynamically displays KPIs, project estimations, and resource mix suggestions.", "EffortsInDays": "8"}, {"Level": "3", "Category": "DESIGN", "Subcategory": "UI/UX Development", "UserStory": "Design a detailed profile view for resources including links to LinkedIn profiles and credit scores.", "EffortsInDays": "4"}, {"Level": "4", "Category": "DESIGN", "Subcategory": "UI/UX Development", "UserStory": "Develop a system for displaying notifications, profiles, and settings in a user-friendly manner.", "EffortsInDays": "6"}, {"Level": "5", "Category": "DESIGN", "Subcategory": "UI/UX Development", "UserStory": "Implement a role-based access control UI for managing user permissions

In [118]:
d = json.loads(d)
d

[{'Level': '1',
  'Category': 'DESIGN',
  'Subcategory': 'UI/UX Development',
  'UserStory': 'Design the login and user registration interface ensuring intuitive user interaction.',
  'EffortsInDays': '5'},
 {'Level': '2',
  'Category': 'DESIGN',
  'Subcategory': 'UI/UX Development',
  'UserStory': 'Create a dashboard layout that dynamically displays KPIs, project estimations, and resource mix suggestions.',
  'EffortsInDays': '8'},
 {'Level': '3',
  'Category': 'DESIGN',
  'Subcategory': 'UI/UX Development',
  'UserStory': 'Design a detailed profile view for resources including links to LinkedIn profiles and credit scores.',
  'EffortsInDays': '4'},
 {'Level': '4',
  'Category': 'DESIGN',
  'Subcategory': 'UI/UX Development',
  'UserStory': 'Develop a system for displaying notifications, profiles, and settings in a user-friendly manner.',
  'EffortsInDays': '6'},
 {'Level': '5',
  'Category': 'DESIGN',
  'Subcategory': 'UI/UX Development',
  'UserStory': 'Implement a role-based access

In [113]:
query = """TASK:
    Your task is to create a comprehensive list of user stories covering all major aspects for ANALYSIS with technical details as well.
    Extract all technical design-related requirements from the document provided, focusing on specifications, architecture, implementation details, system components, interfaces, and performance metrics.
    Please provide atleast 12-15 detailed requirements. Keep CATEGORY column value as ANALYSIS.
    Additionally, provide the estimate the effort (in days) to the best of your knowledge, with some buffer required for each requirement.
   """

first_pass = rag_chain.invoke(query)
final_wbs += first_pass
print(first_pass)

[{'Level': '1', 'Category': 'Analysis', 'Subcategory': 'Technical Requirements', 'UserStory': 'Implement extensive customization features for user personalization.', 'EffortsInDays': '5'}, {'Level': '2', 'Category': 'Analysis', 'Subcategory': 'Integration', 'UserStory': 'Enable seamless data sharing and integration with metaverse platforms.', 'EffortsInDays': '8'}, {'Level': '3', 'Category': 'Analysis', 'Subcategory': 'Feature Development', 'UserStory': 'Develop algorithms for dynamic tree reactions to seasonal changes.', 'EffortsInDays': '10'}, {'Level': '4', 'Category': 'Analysis', 'Subcategory': 'Feature Development', 'UserStory': 'Implement tree care functionalities for user interaction.', 'EffortsInDays': '4'}, {'Level': '5', 'Category': 'Analysis', 'Subcategory': 'Rewards System', 'UserStory': 'Develop a badge and NFT rewards system for user engagement.', 'EffortsInDays': '7'}, {'Level': '6', 'Category': 'Analysis', 'Subcategory': 'Data Management', 'UserStory': 'Implement mechan

In [14]:
query = """TASK:
    Your task is to create a comprehensive list of user stories covering all major aspects for TECHNICAL DESIGN with technical details as well.
    Extract all technical design-related requirements from the document provided, focusing on specifications, architecture, implementation details, system components, interfaces, and performance metrics.
    Please provide atleast 12-15 detailed requirements. Keep CATEGORY column value as TECHNICAL DESIGN.
    Additionally, provide the estimate the effort (in days) to the best of your knowledge, with some buffer required for each requirement.
   """

first_pass = rag_chain.invoke(query)
final_wbs += first_pass
print(first_pass)

```csv
Level,Category,Subcategory,User Story,Effort (Days)
1,TECHNICAL DESIGN,Architecture,"Develop a scalable application architecture to support a large user base and data volume.",15
2,TECHNICAL DESIGN,Security,"Implement robust security measures for data protection and privacy, including encryption and secure APIs.",20
3,TECHNICAL DESIGN,Data Persistence,"Design and implement secure data persistence mechanisms to save user progress across sessions.",10
4,TECHNICAL DESIGN,Performance,"Optimize application performance across different devices and network conditions.",18
5,TECHNICAL DESIGN,Localization,"Support multiple languages and implement accessibility features for users with disabilities.",12
6,TECHNICAL DESIGN,Testing,"Set up a comprehensive testing framework for usability, compatibility, and performance across devices.",25
7,TECHNICAL DESIGN,API Integration,"Implement APIs for seamless data sharing and integration with third-party services like social networks and NFT platform

In [15]:
query = """TASK:
    Your task is to create a comprehensive list of user stories covering all major aspects for FRONTEND DEVELOPMENT with technical details as well.
    Please provide atleast 12-15 detailed requirements. Keep CATEGORY column value as FRONTEND DEVELOPMENT.
    Additionally, provide the estimate the effort (in days) to the best of your knowledge, with some buffer required for each requirement.
   """

first_pass = rag_chain.invoke(query)
final_wbs += first_pass
print(first_pass)

```csv
Level,Category,Subcategory,User Story,Effort (Days)
1,FRONTEND DEVELOPMENT,Interactive Virtual Environment,"Develop a virtual environment for user interaction with nature, including planting and designing forests.",30
2,FRONTEND DEVELOPMENT,User Interface Design,"Design intuitive and user-friendly interfaces for iOS and Android, ensuring UI/UX consistency across devices.",25
3,FRONTEND DEVELOPMENT,Planting and Design Features,"Implement user functionalities for tree selection, planting, and forest design within the virtual environment.",20
4,FRONTEND DEVELOPMENT,Growth Simulation,"Develop growth simulation algorithms for trees considering environmental factors.",18
5,FRONTEND DEVELOPMENT,Badge and NFT Rewards,"Create a reward system with badges or NFTs for user achievements in tree planting.",15
6,FRONTEND DEVELOPMENT,Statistical Data Collection and Sharing,"Implement features for users to collect, share, and display tree-related statistical data.",12
7,FRONTEND DEVELOPMENT,Tree

In [16]:
query = """TASK:
    Your task is to create a comprehensive list of user stories covering all major aspects for BACKEND DEVELOPMENT with technical details as well,  
    including but not limited to server-side functionality, database management, API integration, and performance optimization.
    Please provide atleast 12-15 detailed requirements. Keep CATEGORY column value as BACKEND DEVELOPMENT.
    Additionally, provide the estimate the effort (in days) to the best of your knowledge, with some buffer required for each requirement.
   """

first_pass = rag_chain.invoke(query)
final_wbs += first_pass
print(first_pass)

```csv
Level,Category,Subcategory,User Story,Effort (Days)
1,BACKEND DEVELOPMENT,Database Management,"Develop secure data persistence mechanisms to save user progress and data.",10
2,BACKEND DEVELOPMENT,API Integration,"Integrate with geolocation services for location-based discovery features.",8
3,BACKEND DEVELOPMENT,Server-Side Functionality,"Implement server-side logic for user customization options, including personal and enterprise customization.",12
4,BACKEND DEVELOPMENT,API Integration,"Enable seamless data sharing and integration with metaverse platforms like social networks, blogs, and NFT platforms.",15
5,BACKEND DEVELOPMENT,Server-Side Functionality,"Develop algorithms for dynamic tree interactions based on seasonal and weather changes.",14
6,BACKEND DEVELOPMENT,Server-Side Functionality,"Implement tree care functions allowing users to maintain their trees by feeding, watering, etc.",11
7,BACKEND DEVELOPMENT,Security and Privacy,"Implement robust security measures for user d

In [17]:
query = """TASK:
    Your task is to create a comprehensive list of user stories covering all major aspects for DATABASE DEVELOPMENT with technical details as well.  
    Ensure to include specifics such as database schema design, data modeling, query optimization, database administration, 
    and any other relevant aspects directly related to the development and management of databases.
    Please provide atleast 12-15 detailed requirements. Keep CATEGORY column value as DATABASE DEVELOPMENT.
    Additionally, provide the estimate the effort (in days) to the best of your knowledge, with some buffer required for each requirement.
   """

first_pass = rag_chain.invoke(query)
final_wbs += first_pass
print(first_pass)

```csv
Level,Category,Subcategory,User Story,Effort (Days)
1,DATABASE DEVELOPMENT,Schema Design,"Design a comprehensive database schema to support user data, tree data, and interactions",5
2,DATABASE DEVELOPMENT,Data Modeling,"Model relational and non-relational data structures for efficient data storage and retrieval",4
3,DATABASE DEVELOPMENT,Query Optimization,"Optimize database queries for high performance, focusing on speed and efficiency",3
4,DATABASE DEVELOPMENT,Database Administration,"Set up and configure database servers, ensuring high availability and disaster recovery",7
5,DATABASE DEVELOPMENT,Security,"Implement robust security measures for data protection, including encryption and access controls",6
6,DATABASE DEVELOPMENT,Data Persistence,"Develop mechanisms for secure and efficient data persistence to save user progress and interactions",5
7,DATABASE DEVELOPMENT,API Integration,"Create APIs for seamless data sharing and integration with third-party services",4
8,DATABASE 

In [18]:
query = """TASK:
    Your task is to create a comprehensive list of user stories covering all major aspects for TESTING AND QUIALITY ASSURANCE with technical details as well. 
    Please provide atleast 12-15 detailed requirements. Keep CATEGORY column value as TESTING AND Q/A.
    Additionally, provide the estimate the effort (in days) to the best of your knowledge, with some buffer required for each requirement.
   """

first_pass = rag_chain.invoke(query)
final_wbs += first_pass
print(first_pass)

```csv
Level,Category,Subcategory,User Story,Effort (Days)
1,TESTING AND Q/A,Performance Optimization,"Test application performance across different devices and network conditions.",10
2,TESTING AND Q/A,Performance Optimization,"Analyze and minimize resource usage during peak and off-peak times.",12
3,TESTING AND Q/A,Performance Optimization,"Validate application architecture's ability to handle large user base and data volume.",15
4,TESTING AND Q/A,Localization and Accessibility,"Ensure the application supports multiple languages.",8
5,TESTING AND Q/A,Localization and Accessibility,"Test accessibility features for users with disabilities.",10
6,TESTING AND Q/A,Device and OS Compatibility,"Conduct rigorous testing across various devices, screen sizes, and operating system versions.",14
7,TESTING AND Q/A,Usability,"Perform usability testing to gather feedback and refine user experience.",12
8,TESTING AND Q/A,Compliance,"Ensure compliance with app store guidelines and regulations.",7
9,T

In [19]:
query = """TASK:
    Your task is to create a comprehensive list of user stories covering all major aspects for INFRASTRUCTURE SETUP with technical details as well. 
    Only consider requirements related to infrastructure setup to run this project like VMs, Networks, Databases, IPs, Web Hosting etc.. 
    Please provide atleast 12-15 detailed requirements. Keep CATEGORY column value as INFRASTRUCTURE.
    Additionally, provide the estimate the effort (in days) to the best of your knowledge, with some buffer required for each requirement.
   """

first_pass = rag_chain.invoke(query)
final_wbs += first_pass
print(first_pass)

```csv
Level,Category,Subcategory,User Story,Effort (Days)
1,INFRASTRUCTURE,VM Setup,"Set up virtual machines for development, testing, and production environments.",5
2,INFRASTRUCTURE,Network Configuration,"Configure internal and external network settings, including firewalls and load balancers.",3
3,INFRASTRUCTURE,Database Deployment,"Deploy and configure databases to store user data, tree data, and application logs securely.",4
4,INFRASTRUCTURE,IP Allocation,"Allocate static IP addresses for servers and configure DNS settings.",2
5,INFRASTRUCTURE,Web Hosting,"Select and set up web hosting services for the WordPress website.",3
6,INFRASTRUCTURE,Security,"Implement security measures, including SSL certificates, data encryption, and compliance checks.",5
7,INFRASTRUCTURE,Storage,"Set up cloud storage solutions for data persistence and backups.",3
8,INFRASTRUCTURE,API Gateway,"Configure an API gateway for managing, monitoring, and securing microservices.",4
9,INFRASTRUCTURE,CDN Setup,"S

In [20]:
query = """TASK:
    Your task is to create a comprehensive list of user stories covering all major aspects for DEPLOYMENT AND MONITORING with technical details as well. 
    Only consider requirements related to deployment and monitoring like performance, security, monitoring, clustering, availability etc.
    Please provide atleast 12-15 detailed requirements. Keep CATEGORY column value as DEPLOYMENT.
    Additionally, provide the estimate the effort (in days) to the best of your knowledge, with some buffer required for each requirement.
   """

first_pass = rag_chain.invoke(query)
final_wbs += first_pass
print(first_pass)

```csv
Level,Category,Subcategory,User Story,Effort (Days)
1,DEPLOYMENT,Performance Optimization,"Implement caching mechanisms to reduce load times and improve user experience.",5
2,DEPLOYMENT,Scalability,"Design and implement a scalable architecture to support a growing number of users.",10
3,DEPLOYMENT,Security,"Integrate SSL/TLS for secure data transmission.",3
4,DEPLOYMENT,Security,"Implement robust data encryption for stored user data.",4
5,DEPLOYMENT,Monitoring,"Set up a monitoring system for real-time performance tracking.",5
6,DEPLOYMENT,Monitoring,"Implement logging for error tracking and diagnostics.",4
7,DEPLOYMENT,Availability,"Design a high availability infrastructure to minimize downtime.",7
8,DEPLOYMENT,Clustering,"Implement server clustering to distribute load and improve reliability.",8
9,DEPLOYMENT,Updates and Maintenance,"Develop a process for seamless app updates with minimal downtime.",5
10,DEPLOYMENT,Documentation,"Create comprehensive deployment documentation for

In [21]:
final_wbs

'```csv\nLevel,Category,Subcategory,User Story,Effort (Days)\n1,DESIGN,UI/UX,"Develop intuitive and user-friendly interfaces for iOS and Android devices, ensuring consistency across screen sizes.",15\n2,DESIGN,UI/UX,"Implement extensive customization features for user experience personalization.",12\n3,DESIGN,Interactivity,"Enable seamless data sharing and integration with metaverse platforms like social networks and NFT platforms.",18\n4,DESIGN,Virtual Environment,"Develop a virtual environment for users to interact with nature, including planting and designing forests.",22\n5,DESIGN,Features,"Implement functionalities for selecting, planting, and designing virtual forests with terrain modification and tree placement.",20\n6,DESIGN,Simulation,"Develop algorithms for growth simulation considering environmental conditions.",16\n7,DESIGN,Data Persistence,"Implement secure mechanisms for saving user progress and data.",14\n8,DESIGN,Social Interaction,"Integrate social features for sharing

In [22]:
a = final_wbs

In [77]:
import json
import csv

data = json.loads(final_wbs)

# Define CSV file name
csv_file = "output.csv"

# Write JSON data to CSV file
with open(csv_file, mode='w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=data[0].keys())
    writer.writeheader()
    for row in data:
        writer.writerow(row)

print(f"CSV file '{csv_file}' has been created.")


CSV file 'output.csv' has been created.


In [103]:
d = """[
    {
        "Level": "1",
        "Category": "DESIGN",
        "Subcategory": "UI/UX Design",
        "UserStory": "Design a user-friendly login and registration flow for the platform.",
        "EffortsInDays": "5",
    },
    {
        "Level": "2",
        "Category": "DESIGN",
        "Subcategory": "UI/UX Design",
        "UserStory": "Create a dashboard UI that effectively displays KPIs, project estimations, and resource rankings.",
        "EffortsInDays": "8",
    },
    {
        "Level": "3",
        "Category": "DESIGN",
        "Subcategory": "UI/UX Design",
        "UserStory": "Design mobile-responsive layouts for the entire platform.",
        "EffortsInDays": "10",
    },
    {
        "Level": "4",
        "Category": "DESIGN",
        "Subcategory": "UI/UX Design",
        "UserStory": "Develop a system for displaying notifications, profiles, and settings in a user-friendly manner.",
        "EffortsInDays": "4",
    },
    {
        "Level": "5",
        "Category": "DESIGN",
        "Subcategory": "UI/UX Design",
        "UserStory": "Implement role-based access control UI for managing permissions.",
        "EffortsInDays": "6",
    },
    {
        "Level": "6",
        "Category": "DESIGN",
        "Subcategory": "UI/UX Design",
        "UserStory": "Design the UI for a detailed analytics page showing employee performance metrics.",
        "EffortsInDays": "7",
    },
    {
        "Level": "7",
        "Category": "DESIGN",
        "Subcategory": "UI/UX Design",
        "UserStory": "Create a user interface for code reusability analysis results.",
        "EffortsInDays": "5",
    },
    {
        "Level": "8",
        "Category": "DESIGN",
        "Subcategory": "UI/UX Design",
        "UserStory": "Design the UI for integrating with JIRA, GitHub, and SonarQube for data retrieval.",
        "EffortsInDays": "9",
    },
    {
        "Level": "9",
        "Category": "DESIGN",
        "Subcategory": "UI/UX Design",
        "UserStory": "Develop a UI component for project timeline and cost estimation features.",
        "EffortsInDays": "7",
    },
    {
        "Level": "10",
        "Category": "DESIGN",
        "Subcategory": "UI/UX Design",
        "UserStory": "Design a feedback system for users to submit their reviews on the platform.",
        "EffortsInDays": "3",
    },
    {
        "Level": "11",
        "Category": "DESIGN",
        "Subcategory": "UI/UX Design",
        "UserStory": "Create a UI for displaying the credit scoring of employees.",
        "EffortsInDays": "4",
    },
    {
        "Level": "12",
        "Category": "DESIGN",
        "Subcategory": "UI/UX Design",
        "UserStory": "Design an interactive UI for resource allocation and project management.",
        "EffortsInDays": "8",
    },
    {
        "Level": "13",
        "Category": "DESIGN",
        "Subcategory": "UI/UX Design",
        "UserStory": "Implement a system for visualizing the integration process with external tools.",
        "EffortsInDays": "5",
    },
    {
        "Level": "14",
        "Category": "DESIGN",
        "Subcategory": "UI/UX Design",
        "UserStory": "Design a settings page for configuring the AI-based features of the platform.",
        "EffortsInDays": "4",
    },
    {
        "Level": "15",
        "Category": "DESIGN",
        "Subcategory": "UI/UX Design",
        "UserStory": "Create a dynamic UI for adjusting project parameters and seeing the impact on cost and timelines.",
        "EffortsInDays": "6",
    },
    {
        "Level": "1",
        "Category": "Analysis",
        "Subcategory": "Requirements Gathering",
        "UserStory": "Analyze systems to extract relevant KPIs for the platform.",
        "EffortsInDays": "3",
    },
    {
        "Level": "2",
        "Category": "Analysis",
        "Subcategory": "Integration Requirements",
        "UserStory": "Integrate platform with Bamboo HR to retrieve employee data.",
        "EffortsInDays": "2",
    },
    {
        "Level": "3",
        "Category": "Analysis",
        "Subcategory": "Feature Development",
        "UserStory": "Develop code reusability feature based on BRD input.",
        "EffortsInDays": "5",
    },
    {
        "Level": "4",
        "Category": "Analysis",
        "Subcategory": "AI Model Development",
        "UserStory": "Create AI model to suggest project timelines.",
        "EffortsInDays": "4",
    },
    {
        "Level": "5",
        "Category": "Analysis",
        "Subcategory": "Data Retrieval",
        "UserStory": "Retrieve and process data from Jira for project management.",
        "EffortsInDays": "3",
    },
    {
        "Level": "6",
        "Category": "Analysis",
        "Subcategory": "Security",
        "UserStory": "Implement role-based access controls for the system.",
        "EffortsInDays": "2",
    },
    {
        "Level": "7",
        "Category": "Analysis",
        "Subcategory": "Data Analysis",
        "UserStory": "Fetch and analyze data from SonarQube for code quality metrics.",
        "EffortsInDays": "2",
    },
    {
        "Level": "8",
        "Category": "Analysis",
        "Subcategory": "Project Costing",
        "UserStory": "Develop project costing feature considering resource profiles.",
        "EffortsInDays": "4",
    },
    {
        "Level": "9",
        "Category": "Analysis",
        "Subcategory": "UI/UX Design",
        "UserStory": "Design dashboard for displaying KPIs, rankings, and estimations.",
        "EffortsInDays": "3",
    },
    {
        "Level": "10",
        "Category": "Analysis",
        "Subcategory": "Authentication",
        "UserStory": "Develop system for user authentication and role management.",
        "EffortsInDays": "3",
    },
    {
        "Level": "11",
        "Category": "Analysis",
        "Subcategory": "Integration Requirements",
        "UserStory": "Plan integration with multiple software tools for data collection.",
        "EffortsInDays": "3",
    },
    {
        "Level": "12",
        "Category": "Analysis",
        "Subcategory": "AI Model Development",
        "UserStory": "Develop AI model for project costing based on cloud and infrastructure costs.",
        "EffortsInDays": "5",
    },
    {
        "Level": "13",
        "Category": "Analysis",
        "Subcategory": "Data Analysis",
        "UserStory": "Analyze historical project data for AI model training.",
        "EffortsInDays": "4",
    },
    {
        "Level": "14",
        "Category": "Analysis",
        "Subcategory": "Feature Development",
        "UserStory": "Implement feature for continuous auto-update scores.",
        "EffortsInDays": "4",
    },
    {
        "Level": "15",
        "Category": "Analysis",
        "Subcategory": "Non-Functional Requirements",
        "UserStory": "Ensure system privacy and security of data.",
        "EffortsInDays": "2",
    },
]
"""

#d_valid = d.replace(',\n    }', '\n    }')
json_data = json.dumps(d)
print(json_data)

"[\n    {\n        \"Level\": \"1\",\n        \"Category\": \"DESIGN\",\n        \"Subcategory\": \"UI/UX Design\",\n        \"UserStory\": \"Design a user-friendly login and registration flow for the platform.\",\n        \"EffortsInDays\": \"5\",\n    },\n    {\n        \"Level\": \"2\",\n        \"Category\": \"DESIGN\",\n        \"Subcategory\": \"UI/UX Design\",\n        \"UserStory\": \"Create a dashboard UI that effectively displays KPIs, project estimations, and resource rankings.\",\n        \"EffortsInDays\": \"8\",\n    },\n    {\n        \"Level\": \"3\",\n        \"Category\": \"DESIGN\",\n        \"Subcategory\": \"UI/UX Design\",\n        \"UserStory\": \"Design mobile-responsive layouts for the entire platform.\",\n        \"EffortsInDays\": \"10\",\n    },\n    {\n        \"Level\": \"4\",\n        \"Category\": \"DESIGN\",\n        \"Subcategory\": \"UI/UX Design\",\n        \"UserStory\": \"Develop a system for displaying notifications, profiles, and settings in a us

In [104]:
data = json.loads(json_data)

In [105]:
print(data)

[
    {
        "Level": "1",
        "Category": "DESIGN",
        "Subcategory": "UI/UX Design",
        "UserStory": "Design a user-friendly login and registration flow for the platform.",
        "EffortsInDays": "5",
    },
    {
        "Level": "2",
        "Category": "DESIGN",
        "Subcategory": "UI/UX Design",
        "UserStory": "Create a dashboard UI that effectively displays KPIs, project estimations, and resource rankings.",
        "EffortsInDays": "8",
    },
    {
        "Level": "3",
        "Category": "DESIGN",
        "Subcategory": "UI/UX Design",
        "UserStory": "Design mobile-responsive layouts for the entire platform.",
        "EffortsInDays": "10",
    },
    {
        "Level": "4",
        "Category": "DESIGN",
        "Subcategory": "UI/UX Design",
        "UserStory": "Develop a system for displaying notifications, profiles, and settings in a user-friendly manner.",
        "EffortsInDays": "4",
    },
    {
        "Level": "5",
        "Catego